## Nhóm 11:
**Thành viên:**<br>
*Võ Quốc Huy - 20081001<br>
Nguyễn Quang Bảo - 20083601<br>
Nguyễn Xuân Giang - 20079601*

## Decision tree ID3

In [1]:
# import các thư viện cần thiết
import pandas as pd
import numpy as np
from pprint import pprint

# Giới thiệu data

- Bộ dữ liệu này bao gồm 101 loài động vật từ một vườn thú.<br>
- Có 16 biến với các đặc điểm khác nhau để mô tả các loài động vật.<br>
- Mục đích của tập dữ liệu này là có thể dự đoán thuộc tính của các loài động vật<br>
- Link data: https://www.kaggle.com/datasets/uciml/zoo-animal-classification?select=zoo.csv

**Mô tả**<br>
Thông tin thuộc tính

1. animal_name: Tên<br>
2. hair: lông<br>
3. feathers: lông vũ <br>
4. eggs: trứng<br>
5. milk: sữa<br>
6. airborne: trên không<br>
7. aquatic: thủy sinh<br>
8. predator: Động vật ăn thịt<br>
9. toothed: có răng<br>
10. backbone: xương sống<br>
11. breathes: thở bằng phổi<br>
12. venomous: có nọc độc<br>
13. fins: vây<br>
14. legs Numeric (set of values: {0,2,4,5,6,8}): chân<br>
15. tail: có đuôi<br>
16. domestic: nội địa<br>
17. catsize: <br>
18. class_type Numeric (integer values in range [1,7]): giá trị nguyên<br>

# Khám phá dữ liệu

In [2]:
dataset = pd.read_csv('zoo_data.csv',index_col=0)

In [3]:
dataset.head()

,animal_name,hair,feathers,eggs,milk,airbone,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [4]:
dataset.shape

(101, 18)

In [5]:
# xóa cột animal name
dataset=dataset.drop('animal_name',axis=1)

In [6]:
#kiểm tra dữ liệu nal
dataset.isna().sum()

hair        0
feathers    0
eggs        0
milk        0
airbone     0
aquatic     0
predator    0
toothed     0
backbone    0
breathes    0
venomous    0
fins        0
legs        0
tail        0
domestic    0
catsize     0
class       0
dtype: int64

In [7]:
#xem thông tin bộ dữ liệu
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   hair      101 non-null    int64
 1   feathers  101 non-null    int64
 2   eggs      101 non-null    int64
 3   milk      101 non-null    int64
 4   airbone   101 non-null    int64
 5   aquatic   101 non-null    int64
 6   predator  101 non-null    int64
 7   toothed   101 non-null    int64
 8   backbone  101 non-null    int64
 9   breathes  101 non-null    int64
 10  venomous  101 non-null    int64
 11  fins      101 non-null    int64
 12  legs      101 non-null    int64
 13  tail      101 non-null    int64
 14  domestic  101 non-null    int64
 15  catsize   101 non-null    int64
 16  class     101 non-null    int64
dtypes: int64(17)
memory usage: 14.2 KB


# Model Decision Tree

In [18]:
#tính entropy
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [19]:
#tính thông tin gain
def InfoGain(data,split_attribute_name,target_name="class"):
 
    total_entropy = entropy(data[target_name])
     
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [28]:

def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        #print(data[target_attribute_name])
        return np.unique(data[target_attribute_name])[0]
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    elif len(features) ==0:
        return parent_node_class   
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]  
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] 
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{}}
        features = [i for i in features if i != best_feature]       
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data,originaldata,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
            
        return(tree)

**Dự đoán**

In [29]:
def predict(query,tree,default = 1):   
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
        

In [30]:
dataset.shape

(101, 17)

**Tách dữ liệu tập huấn luyện và tập kiểm thử**

In [31]:
def train_test_split(dataset):
    training_data = dataset.iloc[:80].reset_index(drop=True)
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data
training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 



**Dự đoán với tập dữ liệu kiểm thử**

In [32]:
def test(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    print(len(queries))
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
       # print(predict(queries[i],tree,1.0) )
    return np.sum(((predicted["predicted"] == data["class"])/len(data)))

**Solution**

In [33]:

tree = ID3(training_data,training_data,training_data.columns[:-1])
pprint(tree)
print('\n')
print('The prediction accuracy is: ',test(testing_data,tree))
#test(testing_data,tree)

{'legs': {0: {'fins': {0.0: {'toothed': {0.0: 7.0, 1.0: 3.0}},
                       1.0: {'eggs': {0.0: 1.0, 1.0: 4.0}}}},
          2: {'hair': {0.0: 2.0, 1.0: 1.0}},
          4: {'hair': {0.0: {'toothed': {0.0: 7.0, 1.0: 5.0}}, 1.0: 1.0}},
          6: {'aquatic': {0.0: 6.0, 1.0: 7.0}},
          8: 7.0}}


21
The prediction accuracy is:  0.8571428571428574


In [34]:
# view issue with prediction
print(sorted(training_data['legs'].unique()))
print(sorted(testing_data['legs'].unique()))

[0, 2, 4, 6, 8]
[0, 2, 4, 5, 6]
